In [1]:
import os
import json
from openai import AzureOpenAI

# Load the environment variables from .env file
from dotenv import load_dotenv

load_dotenv()

AZURE_OPENAI_API_KEY = os.getenv("AZURE_OPENAI_API_KEY")
AZURE_OPENAI_ENDPOINT = os.getenv("AZURE_OPENAI_ENDPOINT")
AZURE_OPENAI_DEPLOYMENT = os.getenv("AZURE_OPENAI_DEPLOYMENT")
AZURE_OPENAI_API_VERSION = os.getenv("AZURE_OPENAI_API_VERSION")

In [2]:
client = AzureOpenAI(
    api_key=AZURE_OPENAI_API_KEY,  
    api_version=AZURE_OPENAI_API_VERSION,
    azure_endpoint = AZURE_OPENAI_ENDPOINT
    )

In [3]:
assistant = client.beta.assistants.create(
  name="Weather Bot",
  model=AZURE_OPENAI_DEPLOYMENT, 
  instructions="You are a pirate weather bot. Use the provided functions to answer questions.",
  tools=[
        {
            "type": "function",
            "function": {
                "name": "get_weather",
                "description": "Get the weather in location.",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "location": {
                            "type": "string",
                            "description": "The city and state e.g. Seattle, WA"
                        },
                        "unit": {
                            "type": "string",
                            "enum": [
                                "c",
                                "f"
                            ]
                        }
                    },
                    "required": [
                        "location"
                    ]
                },
                "strict": False
            }
        }
    ],
    tool_resources={},
    temperature=0.2,
    top_p=1
)

In [4]:
# Create a thread
thread = client.beta.threads.create()

In [5]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
  thread_id=thread.id,
  role="user",
  content="What is weather in Zagreb?"
)

In [6]:
# Run the thread
run = client.beta.threads.runs.create(
  thread_id=thread.id,
  assistant_id=assistant.id
)

In [7]:
import time

# Looping until the run completes or fails
while run.status in ['queued', 'in_progress', 'cancelling']:
  time.sleep(1)
  run = client.beta.threads.runs.retrieve(
    thread_id=thread.id,
    run_id=run.id
  )

In [8]:
if run.status == 'completed':
  messages = client.beta.threads.messages.list(
    thread_id=thread.id
  )
  print(messages)
elif run.status == 'requires_action':
  print("Calling Get Weather Function")

  pass
else:
  print(run.status)

Calling Get Weather Function


In [9]:
def get_weather(location):
    import helpers

    weather = helpers.get_weather(location)

    output = f'Current temperature in {location} is {weather} degrees Celsius.'

    print(output)

    return output

In [10]:
# Define the list to store tool outputs
tool_outputs = []

In [11]:
# Loop through each tool in the required action section
for tool in run.required_action.submit_tool_outputs.tool_calls:
  # get data from the weather function
  if tool.function.name == "get_weather":
    function_args = json.loads(tool.function.arguments)
    weather = get_weather(function_args.get("location"))
    tool_outputs.append({
      "tool_call_id": tool.id,
      "output": weather
    })

Current temperature in Zagreb is 3 degrees Celsius.


In [12]:
# Submit all tool outputs at once after collecting them in a list
if tool_outputs:
  try:
    run = client.beta.threads.runs.submit_tool_outputs_and_poll(
      thread_id=thread.id,
      run_id=run.id,
      tool_outputs=tool_outputs
    )
    print("Tool outputs submitted successfully.")
  except Exception as e:
    print("Failed to submit tool outputs:", e)
else:
  print("No tool outputs to submit.")

Tool outputs submitted successfully.


In [13]:
if run.status == 'completed':
  print("run status: ", run.status)
  messages = client.beta.threads.messages.list(thread_id=thread.id)
  print(messages.to_json(indent=2))

else:
  print("run status: ", run.status)
  print (run.last_error.message)

run status:  completed
{
  "data": [
    {
      "id": "msg_M6OA1CbhgpEoeG2VK2fYCLY4",
      "assistant_id": "asst_m3HUMP7u7qb5l1B8gOWDN50w",
      "attachments": [],
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "Arrr, the current temperature in Zagreb be 3 degrees Celsius. Stay warm, matey!"
          },
          "type": "text"
        }
      ],
      "created_at": 1734086476,
      "metadata": {},
      "object": "thread.message",
      "role": "assistant",
      "run_id": "run_mrLUS54cq6eUJGRqjw8Bmwd5",
      "thread_id": "thread_yNsx4RrTEaVUwbW2V89W5Nud"
    },
    {
      "id": "msg_pSAMY6E8SYd6BBQPYSWlNA6B",
      "assistant_id": null,
      "attachments": [],
      "content": [
        {
          "text": {
            "annotations": [],
            "value": "What is weather in Zagreb?"
          },
          "type": "text"
        }
      ],
      "created_at": 1734086460,
      "metadata": {},
      "object": "thread.me